In [61]:
# открываю таблички с тест и трейн
import pandas as pd
train_lemm = pd.read_csv('lemm_train_final.csv')
test_lemm = pd.read_csv('lemm_test_final.csv')

In [101]:
# открываю табличку с y_test
test_score = pd.read_csv('test_labels.csv')

In [103]:
# табличка с теми тестовыми комментами, которые не учитываются 
testnotscore = test_score.loc[test_score['toxic'] == -1]

In [104]:
#дропаю из y_test все то, что не учитывается
test_score.drop(test_score[test_score['toxic'] == -1].index, inplace = True)

In [106]:
#список id, которые не учитываются
notscoreid = testnotscore['id'].to_list()

In [108]:
#дропаю все, что не учитывается, из тестового датасета
test_lemm = test_lemm[~test_lemm['id'].isin(notscoreid)]

In [130]:
test_score.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0


In [144]:
#целевые переменные в тесте
toxic_y_test = test_score['toxic'].to_list()
severe_toxic_y_test = test_score['severe_toxic'].to_list()
obscene_y_test = test_score['obscene'].to_list()
threat_y_test = test_score['threat'].to_list()
insult_y_test = test_score['insult'].to_list()
identity_hate_y_test = test_score['identity_hate'].to_list()

In [137]:
# целевые переменные в трейне
toxic_y_train = train_lemm['toxic'].to_list()
severe_toxic_y_train = train_lemm['severe_toxic'].to_list()
obscene_y_train = train_lemm['obscene'].to_list()
threat_y_train = train_lemm['threat'].to_list()
insult_y_train = train_lemm['insult'].to_list()
identity_hate_y_train = train_lemm['identity_hate'].to_list()

In [113]:
#каунт векторайзер
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
cvec = CountVectorizer(max_features=10000) 
cvec.fit(train_lemm['lemmatized'].values.astype('U'))

X_train_1 = cvec.fit_transform(train_lemm['lemmatized'].values.astype('U'))
X_test_1 = cvec.transform(test_lemm['lemmatized'].values.astype('U'))

In [118]:
#тфидф
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
stops = stopwords.words("english")
import numpy as np

text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=200000)
X_train_text = text_transformer.fit_transform(train_lemm['lemmatized'].values.astype('U'))
X_test_text = text_transformer.transform(test_lemm['lemmatized'].values.astype('U'))

In [151]:
X_test_text.shape

(63978, 200000)

In [124]:
#ворд2век
from gensim.models import Word2Vec

sentences = [sentence.split() for sentence in train_lemm['lemmatized'].values.astype('U')]
w2v_model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

In [126]:
#ворд2век
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train_wv = np.array([vectorize(sentence) for sentence in train_lemm['lemmatized'].values.astype('U')])
X_test_wv = np.array([vectorize(sentence) for sentence in test_lemm['lemmatized'].values.astype('U')])

In [160]:
# общая функция для fit, predict и accuracy по всем метрикам
from sklearn import metrics
import statistics
def toxic_classification(type_of_regression, x_train, x_test):
    logreg_toxic =  type_of_regression.fit(x_train, toxic_y_train)
    logreg_severetox = type_of_regression.fit(x_train, severe_toxic_y_train)
    logreg_obscene = type_of_regression.fit(x_train, obscene_y_train)
    logreg_threat = type_of_regression.fit(x_train, threat_y_train)
    logreg_insult = type_of_regression.fit(x_train, insult_y_train)
    logreg_identity_hate = type_of_regression.fit(x_train, identity_hate_y_train)
    y_pred_toxic = logreg_toxic.predict(x_test)
    y_pred_severetox = logreg_severetox.predict(x_test)
    y_pred_obscene = logreg_obscene.predict(x_test)
    y_pred_threat = logreg_threat.predict(x_test)
    y_pred_insult = logreg_insult.predict(x_test)
    y_pred_identity_hate = logreg_identity_hate.predict(x_test)    
    accuracy_toxic = metrics.accuracy_score(y_pred_toxic, toxic_y_test)
    accuracy_severetox = metrics.accuracy_score(y_pred_severetox, severe_toxic_y_test)
    accuracy_obscene = metrics.accuracy_score(y_pred_obscene, obscene_y_test)
    accuracy_threat = metrics.accuracy_score(y_pred_threat, threat_y_test)
    accuracy_insult = metrics.accuracy_score(y_pred_insult, insult_y_test)
    accuracy_identity_hate = metrics.accuracy_score(y_pred_identity_hate, identity_hate_y_test)
    list_of_preds = [accuracy_toxic, accuracy_severetox, accuracy_obscene, accuracy_threat, accuracy_insult, accuracy_identity_hate]
    return statistics.mean(list_of_preds)

In [161]:
#логрег для каунт векторайзер
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=16, max_iter = 1000)
logreg_toxic =  logreg.fit(X_train_1, toxic_y_train)
toxic_classification(logreg, X_train_1, X_test_1)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

0.9608302854106099

In [163]:
#логрег для тфидф
from sklearn.linear_model import LogisticRegression
tflogreg = LogisticRegression(random_state=16, max_iter = 1000)
toxic_classification(tflogreg, X_train_text, X_test_text)

0.9623620619587984

In [171]:
logreg_toxic =  tflogreg.fit(X_train_text, toxic_y_train)

логрег + тфидф лучшая модель!!!

In [172]:
import pickle
filename = 'toxic_model.sav'
pickle.dump(logreg_toxic, open(filename, 'wb'))

In [164]:
#логрег для ворд2век
from sklearn.linear_model import LogisticRegression
wvlogreg = LogisticRegression(random_state=16, max_iter = 1000)
toxic_classification(wvlogreg, X_train_wv, X_test_wv)

0.9606010399408129

In [165]:
#кнн для каунтвект
from sklearn.neighbors import KNeighborsClassifier
knn_cv = KNeighborsClassifier(n_neighbors=3)
toxic_classification(knn_cv, X_train_1, X_test_1)

0.9608146550376692

In [173]:
#кнн для тфидф
knn_tf = KNeighborsClassifier(n_neighbors=3)
toxic_classification(knn_tf, X_train_text, X_test_text)

KeyboardInterrupt: 